# Generative chatbot dengan menggunakan Neural Network

---







####1.   Install dan import libraries yang diperlukan



In [1]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import string
import numpy as np
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import random
from collections import defaultdict

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/vincentexelcio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/vincentexelcio/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


## Pemrosesan teks


####2.    Membuat fungsi-fungsi yang akan digunakan untuk pemrosesan teks


*   tokenize() (memisahkan kalimat menjadi kumpulan kata)

*   stem() (menghapuskan imbuhan pada kata, contoh having --> have)

*    bag_of_words() (mengubah kalimat menjadi kumpulan kata dan ditandai dengan 0 (kata tidak terdapat pada dataset) dan 1 (kata terdapat pada dataset))



In [2]:
def tokenize(sentence):
    '''
    Fungsi untuk memisahkan kalimat menjadi kumpulan kata
    '''
    return nltk.word_tokenize(sentence)
    

def stem(word):
    '''
    Fungsi untuk menghilangkan imbuhan pada kata
    '''
    stemmer = StemmerFactory().create_stemmer()
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, all_words):
    '''
    Fungsi untuk membuat one-hot encoding list
    '''
    tokenized_sentence = [stem(w) for w in tokenized_sentence]
    bow = np.zeros(len(all_words), dtype = np.float32)
    for idx, word in enumerate(all_words):
        if word in tokenized_sentence:
            bow[idx] = 1.
    return bow

## Memproses dataset untuk training model

####3. Mengambil dataset dan mengolahnya menjadi features dan label



In [3]:
INTENTS_PATH = '../data/intents_indo.json'

with open(INTENTS_PATH, 'r') as f:
    intents = json.load(f)

all_words = [] # untuk menyimpan seluruh kata dari dataset
tags = [] # untuk menyimpan label
pattern_tag = [] # untuk menyimpan pola kalimat beserta labelnya

for intent in intents['intents']:
    # loop dari dataset dan menyimpan label
    tag = intent['tag'].lower()
    tags.append(tag)

    for pattern in intent['patterns']:
        # loop dari tiap pola kalimat yang ada pada label
        ptrn = tokenize(pattern) # memecah kalimat menjadi kumpulan kata
        all_words.extend(ptrn)
        pattern_tag.append((ptrn, tag))

all_words = [stem(word) for word in all_words if word not in string.punctuation] # menghilangkan imbuhan pada kata dan memisahkan tanda baca
all_words = sorted(set(all_words)) # menghapus duplikasi data

x_train = []
y_train = []
for pattern, tag in pattern_tag:
    # loop dari tiap poka kalimat dan label
    bag = bag_of_words(pattern, all_words) # mengubah kalimat menjadi bentuk BoW
    x_train.append(bag)

    label = tags.index(tag) # mencari lokasi index dari label
    y_train.append(label)

x_train = np.array(x_train)
y_train = np.array(y_train)

####4.    Features dan label yang telah dibuat disimpan ke dataset dan dibagi menjadi batch yang berisi 8 data

In [4]:
class ChatDataset(Dataset):
    '''
    Class untuk membuat dataset dari data yang telah diproses
    '''
    def __init__(self):
        self.n_samples = len(x_train)
        self.x_data = x_train
        self.y_data = y_train

    def __getitem__(self, idx):
        return self.x_data[idx], self.y_data[idx]

    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset = dataset, batch_size = 8, shuffle = True, num_workers = 0, )

## Neural Network model

####5.  Membuat model untuk prediksi dengan 3 linear layer dan 1 activation layer menggunakan ReLU

In [5]:
class NeuralNet(nn.Module):
    '''
    Model untuk melakukan prediksi dengan menggunakan 3 layer neural network
    '''
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)

        return out

####6.  Set hyper parameter untuk model

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_size = len(all_words) # input layer memiliki ukuran data yang kita miliki
hidden_size = 10 # hidden size jumlah node di tiap hidden layer
output_size = len(tags) # output layer yang berisi jumlah tag dari dataset
model = NeuralNet(input_size, hidden_size, output_size) # model untuk memberikan tanggapan dari input yang diberikan user

In [7]:
num_epochs = 600 # jumlah iterasi untuk melatih model
learning_rate = 0.001 # skala untuk memperbarui nilai dari parameter saat backward propagation

criterion = nn.CrossEntropyLoss() # untuk perhitungan nilai loss dari prediksi yang dilakukan oleh model
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) # untuk optimisasi nilai parameter pada model

for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(device)

        pred = model(words)
        loss = criterion(pred, labels) # membandingkan prediksi dengan label sebenarnya

        # optimisasi nilai parameter
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # cetak nilai loss tiap iterasi ke 100 dan terakhir
    if (epoch + 1) % 100 == 0:
        print(f'epoch {epoch + 1} / {num_epochs}, loss = {loss.item():.4f}')
print(f'final loss, loss = {loss.item():.4f}')

epoch 100 / 600, loss = 0.6503
epoch 200 / 600, loss = 0.0071
epoch 300 / 600, loss = 0.0072
epoch 400 / 600, loss = 0.0030
epoch 500 / 600, loss = 0.0004
epoch 600 / 600, loss = 0.0004
final loss, loss = 0.0004


####7. Menyimpan model yang telah dilatih beserta dengan parameternya

In [8]:
data = {
    'model_state' : model.state_dict(),
    'input_size' : input_size,
    'hidden_size' : hidden_size,
    'output_size' : output_size,
    'all_words' : all_words,
    'tags' : tags
}

FILE = '../model/trained_model_indo.pth'
torch.save(data, FILE)

print(f'training_complete. file saved to {FILE}')

training_complete. file saved to ../model/trained_model_indo.pth


In [9]:
# import model yang telah dilatih dan dataset yang digunakan

data = torch.load(FILE)
input_size = data['input_size']
hidden_size = data['hidden_size']
output_size = data['output_size']
all_words = data['all_words']
tags = data['tags']
model_state = data['model_state']

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

NeuralNet(
  (l1): Linear(in_features=63, out_features=10, bias=True)
  (l2): Linear(in_features=10, out_features=10, bias=True)
  (l3): Linear(in_features=10, out_features=13, bias=True)
  (relu): ReLU()
)

####8. Chatbot siap digunakan

In [10]:
# bot_name = 'aiBot'
# print("ketik 'quit' untuk keluar dari program chatbot")
# while True:
#     sentence = input('Anda  : ')
#     if sentence == 'quit':
#         print(f'{bot_name} : Terimakasih telah menggunakan chatbot kami.')
#         break

#     sentence = tokenize(sentence)
#     X = bag_of_words(sentence, all_words)
#     X = X.reshape(1, X.shape[0])
#     X = torch.from_numpy(X)

#     output = model(X)
#     _, predicted = torch.max(output, dim = 1)
#     tag = tags[predicted.item()]

#     probs = torch.softmax(output, dim = 1)
#     prob = probs[0][predicted.item()]

#     if prob.item() > 0.55:
#         for intent in intents[ 'intents']:
#             if tag == intent['tag']:
#                 print(f"{bot_name} : {random.choice(intent['responses'])}")
#     else:
#         print(f"{bot_name} : Maaf Saya tidak mengerti")

ketik 'quit' untuk keluar dari program chatbot
aiBot : Terimakasih telah menggunakan chatbot kami.
